# HW 3: Irish or Aussie?
Jasmine Howard and Rishi Parida

## Date Out: Thursday, March 5
## Due Date: Tuesday, March 24 --> Thursday April 9, 2020

This programming assignment aims to help you prepare for the final project in this course. It tasks you to:

* download two large text datasets
* pre-process them into a common format
* divide them into appropriate training and testing sets
* learn a naive bayes classifier on the training set
* iterate and refine the model using a dev-test set
* give a final evaluation of the model using a test set

<u>You may work in teams of two or three (2-tuples or 3-tuples?) for this assignment.</u>

<hr>

This _never-seen-before_ assignment was motivated by our recent discussion of the NLTK Names Corpus and Jurafsky and Martin's example of a toy-classifier to distinguish between "Japan" and "Chinese".

It was a bit difficult finding datasets of the size and detail that I wanted to assign.

Before I settled on the classification task in this assignment __"Irish or Aussie? (You will train a model that distinguishes between newspaper headlines from an Austrailian newspaper compared to an Irish newspaper.)__, I attempted to find good datasets on:
* Jeopardy puzzle clues vs. Who Wants to Be a Millionaire clues
* Peanuts comic strip text vs. Garfield
* Seinfeld sitcom text vs. Friends
* Wine reviews vs. Beer reviews
* ...

If you want, feel free to use different, but comparable datasets for this assignment.

In [17]:
import nltk
import csv
from nltk.tokenize import sent_tokenize, word_tokenize
import random
nltk.download('punkt')
import pickle
from nltk.probability import FreqDist, sum_logs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Task #1

<u>Download the Aussie and Irish headline datasets.</u>

I found the datasets here:
https://en.wikipedia.org/wiki/List_of_datasets_for_machine-learning_research

Look at the "text data" section. There, you'll see citation hyperlinks 168 and 170, which will take you to a kaggle page where you can download the data:
1. A Million News Headlines (these are the headlines for Australia -- Aussie)
2. The Irish Times (Irish)

**Report Entry**
---> The datasets used  where A Million News Headlines (these are the headlines for Australia -- Aussie) and 
The Irish Times (Irish). 

In [0]:
irish = '/content/drive/My Drive/CSC 577/ireland-news-headlines.csv'
aussie = '/content/drive/My Drive/CSC 577/abcnews-date-text.csv'

### Task #2

<u>Process the downloaded <code>csv</code> files in python.</u>

Note that the format in the two datasets are different. Throw out any columns that you don't need. 

Perhaps normalize the data and lowercase everything. Observe that one dataset has capitalization differences. 

Transform the data into some python data structure that also captures whether the headline is Irish or Aussie. I recommend reviewing how we used the tuple `(instance, class)` representation when analyzing the Names Corpus.

In [0]:
# PYTHON CODE HERE

**Report Entry**---> When we imported the CSV files for Irish and Aussie we stored each item of the row ['headline_text'] tupled with place the healines were for. After tupling we created two spearte list with the headlines for irish and aussie. We also decide the lowercase all headlines in both dats sets because we thought it would improve the model. 

In [0]:
aussie_titles = []
with open(aussie, newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        aussie_titles.append((word_tokenize(row['headline_text']), "aussie"))

aussie_titles = [([w.lower() for w in x], y) for (x,y) in aussie_t]

In [0]:
irish_titles = []
with open(irish, newline='', encoding='utf8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        irish_titles.append((word_tokenize(row['headline_text']), "irish"))
irish_titles =[([w.lower() for w in x], y) for (x,y) in irish_t]

**Report Entry** ---> The Irish and Aussie files were so large we decided to pickle the headline/location tuples. so when we needed to used the data later we wouldn’t need to wait as long for the data to become usable. After loading the pickled file we realized the files were different sizes so we thought it would be good to have even data sets so we can have a high probability of quality identifying a dataset.  

In [0]:
a_t = open("/content/drive/My Drive/CSC 577/aussie_titles.p", "wb")
pickle.dump(aussie_titles, a_t)
a_t.close()

i_t = open("/content/drive/My Drive/CSC 577/irish_titles.p", "wb")
pickle.dump(irish_titles, i_t)
i_t.close()

In [0]:
a_t_open = open("/content/drive/My Drive/CSC 577/aussie_titles.p", "rb")
aussie_t = pickle.load(a_t_open)
a_t_open.close()

i_t_open = open("/content/drive/My Drive/CSC 577/irish_titles.p", "rb")
irish_t = pickle.load(i_t_open)
i_t_open.close()

In [20]:
aussie_t = aussie_t[:50000]
irish_t = irish_t[:50000]
print((aussie_t[:25]))
print((irish_t[:25]))

[(['aba', 'decides', 'against', 'community', 'broadcasting', 'licence'], 'aussie'), (['act', 'fire', 'witnesses', 'must', 'be', 'aware', 'of', 'defamation'], 'aussie'), (['a', 'g', 'calls', 'for', 'infrastructure', 'protection', 'summit'], 'aussie'), (['air', 'nz', 'staff', 'in', 'aust', 'strike', 'for', 'pay', 'rise'], 'aussie'), (['air', 'nz', 'strike', 'to', 'affect', 'australian', 'travellers'], 'aussie'), (['ambitious', 'olsson', 'wins', 'triple', 'jump'], 'aussie'), (['antic', 'delighted', 'with', 'record', 'breaking', 'barca'], 'aussie'), (['aussie', 'qualifier', 'stosur', 'wastes', 'four', 'memphis', 'match'], 'aussie'), (['aust', 'addresses', 'un', 'security', 'council', 'over', 'iraq'], 'aussie'), (['australia', 'is', 'locked', 'into', 'war', 'timetable', 'opp'], 'aussie'), (['australia', 'to', 'contribute', '10', 'million', 'in', 'aid', 'to', 'iraq'], 'aussie'), (['barca', 'take', 'record', 'as', 'robson', 'celebrates', 'birthday', 'in'], 'aussie'), (['bathhouse', 'plans', '

### Task #3

<u>Divide your data into a training set and testing set.</u>

No peaking at the test set here on out!

Perhaps even divide your training set into a dev-test set.

In [0]:
# PYTHON CODE HERE

 **Report Entry** -->We combined all the headlines for both the Irish and Aussie datasets. We then shuffled the headlines to ensure the data set is randomly ordered to produce the best training, dev, and testing sets. Combined there were **100000** datelines used in the creation of the model. We intended on using the full dataset however the RAM kept running out so we needed to make changes. The training set held the first **50000** headlines, the dev set held **100** datelines, and the testing set held the last **50000**.

In [21]:
all_titles = []
for a in aussie_t:
    all_titles.append(a)
    
for c in irish_t:
    all_titles.append(c)
    
random.shuffle(all_titles)  
print(all_titles[1])
print(len(all_titles))

(['self', 'representation', 'under', 'review', 'for', 'high', 'court'], 'aussie')
100000


In [0]:
training_set = all_titles[:50000]
dev_set = all_titles[49900:50000]
testing_set = all_titles[50000:]

### Task #4

<u>Learn a naive bayes classifier on the training set.</u>

(For this step you may find it easier to deviate from the Names corpus classifier example, and instead manually compute probabilities following the intuition presented in the "China"/"Japan" J&M example.)

All we really need here are: (1) prior probabilities and (2) conditional probabilities.

 **Report Entry** --> The main headline feature is based off the most common words in both the Irish and Aussie dataset combined. We used the most common **3000** words. We decided to go this rout due to some difficulties with calculating the conditional probabilities. However we were able to successfully calculate the prior probabilities for Irish and Aussie.  

In [14]:
all_words = []
for (a,b) in all_titles:
    all_words.extend(a)


all_words = nltk.FreqDist(all_words)
print(all_words.most_common(15))
print(all_words["bad"])
print(len(all_words))

[('to', 21337), ('in', 13892), ('for', 12878), ('of', 10225), ('on', 8037), ('the', 6748), (';', 3785), ('over', 3602), ('a', 3425), ('and', 3039), ('at', 3004), ("'s", 3003), ('new', 2698), ('with', 2632), ("'", 2570)]
107
35937


In [16]:


def title_features(titles):
  word_features = list(all_words.keys())[:3000]
    words = set(titles)
    features = {}
    for w in word_features:
        features[w] = (w in words) 
    return features
print((title_features(['chamber', 'highlights', 'esperance', 'business', 'woes'])))

#featuresets = [(find_features(titles), place) for (titles, place) in training_set]

{'lloyd': False, 'boots': False, 'bombers': False, 'to': False, 'victory': False, 'spring': False, 'launches': False, 'party': False, "'s": False, 'drive': False, 'on': False, 'referendum': False, 'with': False, 'assurances': False, 'bail': False, 'minardi': False, 'withdraw': False, 'threat': False, 'british': False, 'grand': False, 'prix': False, 'leisel': False, 'leads': False, 'breast': False, 'stroke': False, 'field': False, 'diana': False, 'drama': False, 'is': False, 'ridiculed': False, 'in': False, 'us': False, 'rangers': False, 'demolish': False, 'hearts': False, 'regain': False, 'top': False, 'spot': False, 'opposition': False, 'leader': False, 'wants': False, 'recognition': False, 'final': False, 'doom': False, 'png': False, 'governor': False, 'general': False, 'urges': False, 'election': False, 'unity': False, 'man': False, 'killed': False, 'after': False, 'griffith': False, 'accident': False, 'ispcc': False, 'criticises': False, 'abuse': False, 'document': False, 'ruc': Fa

In [0]:
training_set = [(title_features(titles), place) for (titles, place) in training_set]
dev_set = [(title_features(titles), place) for (titles, place) in dev_set]
testing_set = [(title_features(titles), place) for (titles, place) in testing_set]

In [0]:
#Prior probabilities = number of examples used in training/ whole training set

def prior_prob(training):
  aussie_count = 0
  irish_count = 0
  for (x,y) in training:
    if y == 'aussie':
      aussie_count += 1
    elif y == 'irish':
      irish_count += 1
  aussie_prior = aussie_count / len(training) *100
  irish_prior = irish_count / len(training) *100
  print('Aussie Prior Probability = {}'.format(aussie_prior))
  print('Irish Prior Probability = {}'.format(irish_prior))

prior_prob(training_set)


Aussie Prior Probability = 50.168
Irish Prior Probability = 49.832


In [0]:
# conditional probabilities
import math
def cond_prob(training, all_aussie_words, all_irish_words):
  word = []
  irish_words = []
  aussie_words = []
  aussie_counts = FreqDist(all_aussie_words)
  #for key in aussie_counts:
    #aussie_counts[key] = aussie_counts[key] + 1
  
  irish_counts = FreqDist(all_irish_words)
  #for key in irish_counts:
    #irish_counts[key] = irish_counts[key] + 1

  x,y = training
  #for (x,y) in training:
  if y == 'aussie':
    for z in x:
      aussie_words.append(z)
  elif y == 'irish':
    for a in x:
      irish_words.append(a)  
  
  len_train= len((all_aussie_words + all_irish_words))
  len_irish = len(irish_words)
  len_aussie = len(aussie_words)
  aussie_prob = {}
  irish_prob = {}
  for (key,value) in aussie_counts.items():
    aussie_prob[key]= value / (len_aussie + len_train) *100
  

  for (key,value) in irish_counts.items():
    irish_prob[key]= value / (len_irish + len_train) *100


  aussieVal = aussie_prob.values()
  aussieVal = sum_logs([(x) for x in aussieVal])
  irishVal = irish_prob.values()
  irishVal = sum_logs([(x) for x in irishVal])



  #print(aussie_prob)
  #print(irish_prob)
  return(aussieVal, irishVal)


def just_words(training):
  aussie_word = []
  irish_word = []
  aussie_words = []
  irish_words = []

  
  for x,y in training:
    if y == 'aussie':
      for z in x:
        aussie_words.append(z)
    elif y == 'irish':
      for a in x:
        irish_words.append(a)


  aussie_word.extend(aussie_words)
  irish_word.extend(irish_words)
  aussie_word.extend([x for x in set(irish_words) if x not in aussie_words])
  irish_word.extend([x for x in set(aussie_words) if x not in irish_words])
  
  return (aussie_word, irish_word)




def evaluate(training):
  prob_ans= []
  guess = []

  aussie_words, irish_words = just_words(training)

  for k in training:
    aussie_prob, irish_prob = cond_prob( k,aussie_words, irish_words)
    aussie_prob = (aussie_prob * aussie_prior)
    irish_prob = irish_prob * irish_prior
    if aussie_prob > irish_prob:
      guess.append("aussie")
    else:
      guess.append("irish")
  for (title,answer) in training: 
    prob_ans.append((title,guess[0], answer))
    print(prob_ans)

#evaluate([(['childcare', 'centre', 'staff', 'vow', 'to','to', 'keep', 'centre', 'open'], 'aussie'), (['thatcher', 'prescribes', 'for', 'to', 'hong', 'kong'], 'irish'), (['knights', 'lose', 'parsons', 'and', 'kennedy'], 'aussie')])
evaluate(dev_set)

### Task #5

<u>Iterate and refine the model using a dev-test set.</u>
                        
Let's try to make our model better. What instances is your model misclassifying? Report lessons learned either here, or at the bottom of this notebook. 

**Report Entry** --> Based on out feature set we were about to obtain a guessing accuracy of **80%** using the Naive Bays Algorithm. With a classifier's baseline of 50% for Aussie and 49% for Irish. We were also about to look at some headlines our dataset got wrong. Ways we could have fitted our model is taking out the special characters and stopwords. Also we don’t believe this had a significant impact we kept the model the same.

In [27]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Naive Bayes Algo Accuracy:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

Naive Bayes Algo Accuracy: 80.884
Most Informative Features
                    govt = True           aussie : irish  =    518.7 : 1.0
                     000 = True            irish : aussie =    216.4 : 1.0
                     nsw = True           aussie : irish  =    193.7 : 1.0
              indigenous = True           aussie : irish  =    104.7 : 1.0
                    cork = True            irish : aussie =     92.6 : 1.0
               melbourne = True           aussie : irish  =     71.4 : 1.0
                  aussie = True           aussie : irish  =     62.1 : 1.0
                  dublin = True            irish : aussie =     61.7 : 1.0
                  sydney = True           aussie : irish  =     57.2 : 1.0
                solomons = True           aussie : irish  =     50.8 : 1.0
                    bush = True           aussie : irish  =     49.6 : 1.0
                 aussies = True           aussie : irish  =     48.2 : 1.0
                     emu = True         

In [33]:
errors = []
for (title, place) in dev_set:
    guess = classifier.classify(title_features(title))
    if guess != place:
        errors.append( (title, guess, place) )
print(errors)

[({'lloyd': False, 'boots': False, 'bombers': False, 'to': False, 'victory': False, 'spring': False, 'launches': False, 'party': False, "'s": False, 'drive': False, 'on': False, 'referendum': False, 'with': False, 'assurances': False, 'bail': False, 'minardi': False, 'withdraw': False, 'threat': False, 'british': False, 'grand': False, 'prix': False, 'leisel': False, 'leads': False, 'breast': False, 'stroke': False, 'field': False, 'diana': False, 'drama': False, 'is': False, 'ridiculed': False, 'in': False, 'us': False, 'rangers': False, 'demolish': False, 'hearts': False, 'regain': False, 'top': False, 'spot': False, 'opposition': False, 'leader': False, 'wants': False, 'recognition': False, 'final': False, 'doom': False, 'png': False, 'governor': False, 'general': False, 'urges': False, 'election': False, 'unity': False, 'man': False, 'killed': False, 'after': False, 'griffith': False, 'accident': False, 'ispcc': False, 'criticises': False, 'abuse': False, 'document': False, 'ruc': 

### Task #6

<u>Evaluate your model on the held out test set.</u>
                        
Which metric is most appropriate? Accuracy?

Is there anything else that could be useful to calculate? 

What is the classifier's baseline?

In [0]:
# PYTHON CODE HERE

### Report

Write a technical report (in this Jupyter Notebook, with good Markdown formatting) that documents your findings, "lessons learned", any areas of where you ran into difficult, and also any other interesting details. Include in your report the following details.

The format of the report is up to you, and you may include this information either above, or right here. Use any appropriate layout that best conveys your narrative.

Also submit this python notebook `.ipynb` to D2L.

In [0]:
# PYTHON CODE AND REPORT HERE

**Written throughout the notebook.**